In [1]:
import re
import numpy as np
import pandas as pd

In [2]:
def func_1(x):
    try:
        return float(x)
    except ValueError:
        return np.nan

def find_rating_llama(x):
    response = re.findall(r'Response: (.*)$', x)
    if response:
        rating = re.findall(r'\d', response[0])
        if rating:
            return func_1(rating[0])
        else:
            return np.nan
    else:
        return np.nan
    
def find_rating_tulu(x):
    response = re.findall(r'<\|assistant\|> \n(.*)$', x)
    if response:
        rating = re.findall(r'\d', response[0])
        if rating:
            return func_1(rating[0])
        else:
            return np.nan
    else:
        return np.nan

# Wide Table

## Offensiveness

In [15]:
offensive = pd.read_csv("../data/offensiveness.csv")
offensive_xxl = pd.read_csv("offensiveness_results_flant5xxl.csv")
offensive_ul2 = pd.read_csv("offensiveness_results_flanul2.csv")
offensive_gpt35 = pd.read_table("gpt35_offensiveness.df.tsv")
offensive_gpt4 = pd.read_table("gpt4_offensiveness.df.tsv")

In [20]:
offensive_llama2_p = pd.read_csv("offensiveness_predictions_p1_llama2.csv")
offensive_llama2 = offensive_llama2_p.applymap(find_rating_llama)
offensive_llama2.to_csv("offensiveness_results_p1_llama2.csv", index=False)

In [14]:
offensive_tulu2_7b_p = pd.read_csv("offensiveness_predictions_p1_tulu2_7b.csv")
offensive_tulu2_7b = offensive_tulu2_7b_p.applymap(find_rating_tulu)
offensive_tulu2_7b.to_csv("offensiveness_results_p1_tulu2_7b.csv", index=False)

offensive_tulu2_13b_p = pd.read_csv("offensiveness_predictions_p1_tulu2_13b.csv")
offensive_tulu2_13b = offensive_tulu2_13b_p.applymap(find_rating_tulu)
offensive_tulu2_13b.to_csv("offensiveness_results_p1_tulu2_13b.csv", index=False)

In [16]:
offensive = offensive[['instance_id', 'text', 'label', 'man_score', 'woman_score', 'white_score', 'black_score', 'asian_score']]

offensive_xxl = (offensive_xxl[['a person', 'a male', 'a female', 'a white person', 'a black person', 'an Asian person']]
                .rename(columns={'a person':'flant5_xxl_base',
                                 'a male':'flant5_xxl_male',
                                 'a female':'flant5_xxl_female',
                                 'a white person':'flant5_xxl_white',
                                 'a black person':'flant5_xxl_black',
                                 'an Asian person':'flant5_xxl_asian'}))
offensive_ul2 = (offensive_ul2[['a person', 'a male', 'a female', 'a white person', 'a black person', 'an Asian person']]
                .rename(columns={'a person':'flan_ul2_base',
                                 'a male':'flan_ul2_male',
                                 'a female':'flan_ul2_female',
                                 'a white person':'flan_ul2_white',
                                 'a black person':'flan_ul2_black',
                                 'an Asian person':'flan_ul2_asian'}))
offensive_tulu2_7b = (offensive_tulu2_7b[['a person', 'a male', 'a female', 'a white person', 'a black person', 'an Asian person']]
                .rename(columns={'a person':'tulu2_7b_base',
                                 'a male':'tulu2_7b_male',
                                 'a female':'tulu2_7b_female',
                                 'a white person':'tulu2_7b_white',
                                 'a black person':'tulu2_7b_black',
                                 'an Asian person':'tulu2_7b_asian'}))
offensive_tulu2_13b = (offensive_tulu2_13b[['a person', 'a male person', 'a female person', 'a white person', 'a black person', 'an Asian person']]
                .rename(columns={'a person':'tulu2_13b_base',
                                 'a male person':'tulu2_13b_male',
                                 'a female person':'tulu2_13b_female',
                                 'a white person':'tulu2_13b_white',
                                 'a black person':'tulu2_13b_black',
                                 'an Asian person':'tulu2_13b_asian'}))
offensive_results_w = (offensive
                       .merge(offensive_xxl, how='left', left_index=True, right_index=True)
                       .merge(offensive_ul2, how='left', left_index=True, right_index=True)
                       .merge(offensive_tulu2_7b, how='left', left_index=True, right_index=True)
                       .merge(offensive_tulu2_13b, how='left', left_index=True, right_index=True))

offensive_gpt35['var'] = np.select(
    [
        offensive_gpt35['prefix']=='a person',
        offensive_gpt35['prefix']=='a male',
        offensive_gpt35['prefix']=='a female',
        offensive_gpt35['prefix']=='a White person',
        offensive_gpt35['prefix']=='a Black person',
        offensive_gpt35['prefix']=='an Asian person'
    ],
    ['gpt35_base', 'gpt35_male', 'gpt35_female', 'gpt35_white', 'gpt35_black', 'gpt35_asian'],
    default=pd.NA)
offensive_gpt35 = (offensive_gpt35[['idx', 'var', 'score']]
              .dropna(subset=['var'])
              .pivot(index='idx', columns='var', values='score'))

offensive_gpt4['var'] = np.select(
    [
        offensive_gpt4['prefix']=='a person',
        offensive_gpt4['prefix']=='a male',
        offensive_gpt4['prefix']=='a female',
        offensive_gpt4['prefix']=='a White person',
        offensive_gpt4['prefix']=='a Black person',
        offensive_gpt4['prefix']=='an Asian person'
    ],
    ['gpt4_base', 'gpt4_male', 'gpt4_female', 'gpt4_white', 'gpt4_black', 'gpt4_asian'],
    default=pd.NA)
offensive_gpt4 = (offensive_gpt4[['idx', 'var', 'score']]
              .dropna(subset=['var'])
              .pivot(index='idx', columns='var', values='score'))

offensive_results_w = (offensive_results_w
                       .merge(offensive_gpt35[['gpt35_base', 'gpt35_male', 'gpt35_female', 'gpt35_white', 'gpt35_black', 'gpt35_asian']],
                              how="left", left_index=True, right_on='idx')
                       .merge(offensive_gpt4[['gpt4_base', 'gpt4_male', 'gpt4_female', 'gpt4_white', 'gpt4_black', 'gpt4_asian']],
                              how="left", left_index=True, right_on='idx'))
offensive_results_w.to_csv("../results/offensive_results_w.tsv", sep="\t")

### Wide Table 2

In [8]:
offensive_results_w = pd.read_table(f'../results/offensive_results_w.tsv')
offensive_llama31 = pd.read_csv("offensiveness_results_p1_llama31_8b.csv")
offensive_mistral03 = pd.read_csv("offensiveness_results_p1_mistral03_7b.csv")
offensive_qwen25 = pd.read_csv("offensiveness_results_p1_qwen25_7b.csv")

In [9]:
offensive_llama31 = (offensive_llama31[['a person', 'a male', 'a female', 'a White person', 'a Black person', 'an Asian person']]
                .rename(columns={'a person':'llama31_8b_base',
                                 'a male':'llama31_8b_male',
                                 'a female':'llama31_8b_female',
                                 'a White person':'llama31_8b_white',
                                 'a Black person':'llama31_8b_black',
                                 'an Asian person':'llama31_8b_asian'}))
offensive_mistral03 = (offensive_mistral03[['a person', 'a male', 'a female', 'a White person', 'a Black person', 'an Asian person']]
                .rename(columns={'a person':'mistral03_7b_base',
                                 'a male':'mistral03_7b_male',
                                 'a female':'mistral03_7b_female',
                                 'a White person':'mistral03_7b_white',
                                 'a Black person':'mistral03_7b_black',
                                 'an Asian person':'mistral03_7b_asian'}))
offensive_qwen25 = (offensive_qwen25[['a person', 'a male', 'a female', 'a White person', 'a Black person', 'an Asian person']]
                .rename(columns={'a person':'qwen25_7b_base',
                                 'a male':'qwen25_7b_male',
                                 'a female':'qwen25_7b_female',
                                 'a White person':'qwen25_7b_white',
                                 'a Black person':'qwen25_7b_black',
                                 'an Asian person':'qwen25_7b_asian'}))

offensive_results_w2 = (offensive_results_w
                       .merge(offensive_llama31, how='left', left_index=True, right_index=True)
                       .merge(offensive_mistral03, how='left', left_index=True, right_index=True)
                       .merge(offensive_qwen25, how='left', left_index=True, right_index=True))
offensive_results_w2.to_csv("../results/offensive_results_w2.tsv", sep="\t", index=False)

## Politeness

In [17]:
polite = pd.read_csv("../data/politeness.csv")
polite_xxl = pd.read_csv("politeness_results_flant5xxl.csv")
polite_ul2 = pd.read_csv("politeness_results_flanul2.csv")
polite_gpt35 = pd.read_table("gpt35_politeness.df.tsv")
polite_gpt4 = pd.read_table("gpt4_politeness.df.tsv")

In [17]:
polite_llama2_p = pd.read_csv("politeness_predictions_p1_llama2.csv")
polite_llama2 = polite_llama2_p.applymap(find_rating_llama)
polite_llama2.to_csv("politeness_results_p1_llama2.csv", index=False)

In [18]:
polite_tulu2_7b_p = pd.read_csv("politeness_predictions_p1_tulu2_7b.csv")
polite_tulu2_7b = polite_tulu2_7b_p.applymap(find_rating_tulu)
polite_tulu2_7b.to_csv("politeness_results_p1_tulu2_7b.csv", index=False)

polite_tulu2_13b_p = pd.read_csv("politeness_predictions_p1_tulu2_13b.csv")
polite_tulu2_13b = polite_tulu2_13b_p.applymap(find_rating_tulu)
polite_tulu2_13b.to_csv("politeness_results_p1_tulu2_13b.csv", index=False)

In [19]:
polite = polite[['instance_id', 'text', 'label', 'man_score', 'woman_score', 'white_score', 'black_score', 'asian_score']]

polite_xxl = (polite_xxl[['a person', 'a male', 'a female', 'a white person', 'a black person', 'an Asian person']]
                .rename(columns={'a person':'flant5_xxl_base',
                                 'a male':'flant5_xxl_male',
                                 'a female':'flant5_xxl_female',
                                 'a white person':'flant5_xxl_white',
                                 'a black person':'flant5_xxl_black',
                                 'an Asian person':'flant5_xxl_asian'}))
polite_ul2 = (polite_ul2[['a person', 'a male', 'a female', 'a white person', 'a black person', 'an Asian person']]
                .rename(columns={'a person':'flan_ul2_base',
                                 'a male':'flan_ul2_male',
                                 'a female':'flan_ul2_female',
                                 'a white person':'flan_ul2_white',
                                 'a black person':'flan_ul2_black',
                                 'an Asian person':'flan_ul2_asian'}))
polite_tulu2_7b = (polite_tulu2_7b[['a person', 'a male', 'a female', 'a white person', 'a black person', 'an Asian person']]
                .rename(columns={'a person':'tulu2_7b_base',
                                 'a male':'tulu2_7b_male',
                                 'a female':'tulu2_7b_female',
                                 'a white person':'tulu2_7b_white',
                                 'a black person':'tulu2_7b_black',
                                 'an Asian person':'tulu2_7b_asian'}))
polite_tulu2_13b = (polite_tulu2_13b[['a person', 'a male person', 'a female person', 'a white person', 'a black person', 'an Asian person']]
                .rename(columns={'a person':'tulu2_13b_base',
                                 'a male person':'tulu2_13b_male',
                                 'a female person':'tulu2_13b_female',
                                 'a white person':'tulu2_13b_white',
                                 'a black person':'tulu2_13b_black',
                                 'an Asian person':'tulu2_13b_asian'}))
polite_results_w = (polite
                    .merge(polite_xxl, how='left', left_index=True, right_index=True)
                    .merge(polite_ul2, how='left', left_index=True, right_index=True)
                    .merge(polite_tulu2_7b, how='left', left_index=True, right_index=True)
                    .merge(polite_tulu2_13b, how='left', left_index=True, right_index=True))

polite_gpt35['var'] = np.select(
    [
        polite_gpt35['prefix']=='a person',
        polite_gpt35['prefix']=='a male',
        polite_gpt35['prefix']=='a female',
        polite_gpt35['prefix']=='a White person',
        polite_gpt35['prefix']=='a Black person',
        polite_gpt35['prefix']=='an Asian person'
    ],
    ['gpt35_base', 'gpt35_male', 'gpt35_female', 'gpt35_white', 'gpt35_black', 'gpt35_asian'],
    default=pd.NA)
polite_gpt35 = (polite_gpt35[['idx', 'var', 'score']]
              .dropna(subset=['var'])
              .pivot(index='idx', columns='var', values='score'))

polite_gpt4['var'] = np.select(
    [
        polite_gpt4['prefix']=='a person',
        polite_gpt4['prefix']=='a male',
        polite_gpt4['prefix']=='a female',
        polite_gpt4['prefix']=='a White person',
        polite_gpt4['prefix']=='a Black person',
        polite_gpt4['prefix']=='an Asian person'
    ],
    ['gpt4_base', 'gpt4_male', 'gpt4_female', 'gpt4_white', 'gpt4_black', 'gpt4_asian'],
    default=pd.NA)
polite_gpt4 = (polite_gpt4[['idx', 'var', 'score']]
              .dropna(subset=['var'])
              .pivot(index='idx', columns='var', values='score'))

polite_results_w = (polite_results_w
                    .merge(polite_gpt35[['gpt35_base', 'gpt35_male', 'gpt35_female', 'gpt35_white', 'gpt35_black', 'gpt35_asian']],
                           how="left", left_index=True, right_on='idx')
                    .merge(polite_gpt4[['gpt4_base', 'gpt4_male', 'gpt4_female', 'gpt4_white', 'gpt4_black', 'gpt4_asian']],
                           how="left", left_index=True, right_on='idx'))
polite_results_w.to_csv("../results/polite_results_w.tsv", sep="\t")

### Wide Table 2

In [11]:
polite_results_w = pd.read_table(f'../results/polite_results_w.tsv')
polite_llama31 = pd.read_csv("politeness_results_p1_llama31_8b.csv")
polite_mistral03 = pd.read_csv("politeness_results_p1_mistral03_7b.csv")
polite_qwen25 = pd.read_csv("politeness_results_p1_qwen25_7b.csv")

In [12]:
polite_llama31 = (polite_llama31[['a person', 'a male', 'a female', 'a White person', 'a Black person', 'an Asian person']]
                .rename(columns={'a person':'llama31_8b_base',
                                 'a male':'llama31_8b_male',
                                 'a female':'llama31_8b_female',
                                 'a White person':'llama31_8b_white',
                                 'a Black person':'llama31_8b_black',
                                 'an Asian person':'llama31_8b_asian'}))
polite_mistral03 = (polite_mistral03[['a person', 'a male', 'a female', 'a White person', 'a Black person', 'an Asian person']]
                .rename(columns={'a person':'mistral03_7b_base',
                                 'a male':'mistral03_7b_male',
                                 'a female':'mistral03_7b_female',
                                 'a White person':'mistral03_7b_white',
                                 'a Black person':'mistral03_7b_black',
                                 'an Asian person':'mistral03_7b_asian'}))
polite_qwen25 = (polite_qwen25[['a person', 'a male', 'a female', 'a White person', 'a Black person', 'an Asian person']]
                .rename(columns={'a person':'qwen25_7b_base',
                                 'a male':'qwen25_7b_male',
                                 'a female':'qwen25_7b_female',
                                 'a White person':'qwen25_7b_white',
                                 'a Black person':'qwen25_7b_black',
                                 'an Asian person':'qwen25_7b_asian'}))

polite_results_w2 = (polite_results_w
                       .merge(polite_llama31, how='left', left_index=True, right_index=True)
                       .merge(polite_mistral03, how='left', left_index=True, right_index=True)
                       .merge(polite_qwen25, how='left', left_index=True, right_index=True))
polite_results_w2.to_csv("../results/polite_results_w2.tsv", sep="\t", index=False)

# Robustness Check Table - Prompts

## Flan-T5, UL2

In [ ]:
offensive = pd.read_csv("../data/offensiveness.csv")
offensive_p1_xxl_1 = pd.read_csv("offensiveness_results_flant5xxl.csv")
offensive_p1_xxl_2 = pd.read_csv("offensiveness_results_p1_flant5xxl.csv")
offensive_p1_ul2_1 = pd.read_csv("offensiveness_results_flanul2.csv")
offensive_p1_ul2_2 = pd.read_csv("offensiveness_results_p1_flanul2.csv")
offensive_p2_xxl = pd.read_csv("offensiveness_results_p2_flant5xxl.csv")
offensive_p2_ul2 = pd.read_csv("offensiveness_results_p2_flanul2.csv")
offensive_p3_xxl = pd.read_csv("offensiveness_results_p3_flant5xxl.csv")
offensive_p3_ul2 = pd.read_csv("offensiveness_results_p3_flanul2.csv")

In [ ]:
offensive = offensive[['instance_id', 'text', 'label', 'man_score', 'woman_score', 'white_score', 'black_score', 'asian_score']]

offensive_p1_xxl_1 = (offensive_p1_xxl_1[['a person', 'a white person', 'a black person', 'an Asian person']]
                    .rename(columns={'a person':'p1_flant5_xxl_base',
                                     'a white person':'p1_flant5_xxl_white',
                                     'a black person':'p1_flant5_xxl_black',
                                     'an Asian person':'p1_flant5_xxl_asian'}))
offensive_p1_xxl_2 = (offensive_p1_xxl_2
                      .rename(columns={'a man':'p1_flant5_xxl_man',
                                       'a woman':'p1_flant5_xxl_woman',
                                       'a male person':'p1_flant5_xxl_male',
                                       'a female person':'p1_flant5_xxl_female',
                                       'a normal person':'p1_flant5_xxl_normal',
                                       'an ordinary person':'p1_flant5_xxl_ordinary',
                                       'an average person':'p1_flant5_xxl_average'}))
offensive_p1_xxl = offensive_p1_xxl_1.merge(offensive_p1_xxl_2, how='left', left_index=True, right_index=True)
offensive_p1_xxl = offensive_p1_xxl[['p1_flant5_xxl_base', 'p1_flant5_xxl_man', 'p1_flant5_xxl_woman', 'p1_flant5_xxl_male', 'p1_flant5_xxl_female',
                                     'p1_flant5_xxl_white', 'p1_flant5_xxl_black', 'p1_flant5_xxl_asian',
                                     'p1_flant5_xxl_normal', 'p1_flant5_xxl_ordinary', 'p1_flant5_xxl_average']]

offensive_p1_ul2_1 = (offensive_p1_ul2_1[['a person', 'a white person', 'a black person', 'an Asian person']]
                    .rename(columns={'a person':'p1_flan_ul2_base',
                                     'a white person':'p1_flan_ul2_white',
                                     'a black person':'p1_flan_ul2_black',
                                     'an Asian person':'p1_flan_ul2_asian'}))
offensive_p1_ul2_2 = (offensive_p1_ul2_2
                      .rename(columns={'a man':'p1_flan_ul2_man',
                                       'a woman':'p1_flan_ul2_woman',
                                       'a male person':'p1_flan_ul2_male',
                                       'a female person':'p1_flan_ul2_female',
                                       'a normal person':'p1_flan_ul2_normal',
                                       'an ordinary person':'p1_flan_ul2_ordinary',
                                       'an average person':'p1_flan_ul2_average'}))
offensive_p1_ul2 = offensive_p1_ul2_1.merge(offensive_p1_ul2_2, how='left', left_index=True, right_index=True)
offensive_p1_ul2 = offensive_p1_ul2[['p1_flan_ul2_base', 'p1_flan_ul2_man', 'p1_flan_ul2_woman', 'p1_flan_ul2_male', 'p1_flan_ul2_female',
                                     'p1_flan_ul2_white', 'p1_flan_ul2_black', 'p1_flan_ul2_asian',
                                     'p1_flan_ul2_normal', 'p1_flan_ul2_ordinary', 'p1_flan_ul2_average']]

offensive_p2_xxl = (offensive_p2_xxl
                    .rename(columns={'a person':'p2_flant5_xxl_base',
                                     'a man':'p2_flant5_xxl_man',
                                     'a woman':'p2_flant5_xxl_woman',
                                     'a male person':'p2_flant5_xxl_male',
                                     'a female person':'p2_flant5_xxl_female',
                                     'a white person':'p2_flant5_xxl_white',
                                     'a black person':'p2_flant5_xxl_black',
                                     'an asian person':'p2_flant5_xxl_asian',
                                     'a normal person':'p2_flant5_xxl_normal',
                                     'an ordinary person':'p2_flant5_xxl_ordinary',
                                     'an average person':'p2_flant5_xxl_average'}))

offensive_p2_ul2 = (offensive_p2_ul2
                    .rename(columns={'a person':'p2_flan_ul2_base',
                                     'a man':'p2_flan_ul2_man',
                                     'a woman':'p2_flan_ul2_woman',
                                     'a male person':'p2_flan_ul2_male',
                                     'a female person':'p2_flan_ul2_female',
                                     'a white person':'p2_flan_ul2_white',
                                     'a black person':'p2_flan_ul2_black',
                                     'an asian person':'p2_flan_ul2_asian',
                                     'a normal person':'p2_flan_ul2_normal',
                                     'an ordinary person':'p2_flan_ul2_ordinary',
                                     'an average person':'p2_flan_ul2_average'}))

offensive_p3_xxl = (offensive_p3_xxl
                    .rename(columns={'a person':'p3_flant5_xxl_base',
                                     'a man':'p3_flant5_xxl_man',
                                     'a woman':'p3_flant5_xxl_woman',
                                     'a male person':'p3_flant5_xxl_male',
                                     'a female person':'p3_flant5_xxl_female',
                                     'a white person':'p3_flant5_xxl_white',
                                     'a black person':'p3_flant5_xxl_black',
                                     'an asian person':'p3_flant5_xxl_asian',
                                     'a normal person':'p3_flant5_xxl_normal',
                                     'an ordinary person':'p3_flant5_xxl_ordinary',
                                     'an average person':'p3_flant5_xxl_average'}))

offensive_p3_ul2 = (offensive_p3_ul2
                    .rename(columns={'a person':'p3_flan_ul2_base',
                                     'a man':'p3_flan_ul2_man',
                                     'a woman':'p3_flan_ul2_woman',
                                     'a male person':'p3_flan_ul2_male',
                                     'a female person':'p3_flan_ul2_female',
                                     'a white person':'p3_flan_ul2_white',
                                     'a black person':'p3_flan_ul2_black',
                                     'an asian person':'p3_flan_ul2_asian',
                                     'a normal person':'p3_flan_ul2_normal',
                                     'an ordinary person':'p3_flan_ul2_ordinary',
                                     'an average person':'p3_flan_ul2_average'}))

offensive_prompts_w = (offensive
                       .merge(offensive_p1_xxl, how='left', left_index=True, right_index=True)
                       .merge(offensive_p1_ul2, how='left', left_index=True, right_index=True)
                       .merge(offensive_p2_xxl, how='left', left_index=True, right_index=True)
                       .merge(offensive_p2_ul2, how='left', left_index=True, right_index=True)
                       .merge(offensive_p3_xxl, how='left', left_index=True, right_index=True)
                       .merge(offensive_p3_ul2, how='left', left_index=True, right_index=True))
offensive_prompts_w.to_csv("../results/offensive_prompts_w.tsv", sep="\t", index=False)

In [ ]:
polite = pd.read_csv("../data/politeness.csv")
polite_p1_xxl_1 = pd.read_csv("politeness_results_flant5xxl.csv")
polite_p1_xxl_2 = pd.read_csv("politeness_results_p1_flant5xxl.csv")
polite_p1_ul2_1 = pd.read_csv("politeness_results_flanul2.csv")
polite_p1_ul2_2 = pd.read_csv("politeness_results_p1_flanul2.csv")
polite_p2_xxl = pd.read_csv("politeness_results_p2_flant5xxl.csv")
polite_p2_ul2 = pd.read_csv("politeness_results_p2_flanul2.csv")
polite_p3_xxl = pd.read_csv("politeness_results_p3_flant5xxl.csv")
polite_p3_ul2 = pd.read_csv("politeness_results_p3_flanul2.csv")

In [ ]:
polite = polite[['instance_id', 'text', 'label', 'man_score', 'woman_score', 'white_score', 'black_score', 'asian_score']]

polite_p1_xxl_1 = (polite_p1_xxl_1[['a person', 'a white person', 'a black person', 'an Asian person']]
                    .rename(columns={'a person':'p1_flant5_xxl_base',
                                     'a white person':'p1_flant5_xxl_white',
                                     'a black person':'p1_flant5_xxl_black',
                                     'an Asian person':'p1_flant5_xxl_asian'}))
polite_p1_xxl_2 = (polite_p1_xxl_2
                      .rename(columns={'a man':'p1_flant5_xxl_man',
                                       'a woman':'p1_flant5_xxl_woman',
                                       'a male person':'p1_flant5_xxl_male',
                                       'a female person':'p1_flant5_xxl_female',
                                       'a normal person':'p1_flant5_xxl_normal',
                                       'an ordinary person':'p1_flant5_xxl_ordinary',
                                       'an average person':'p1_flant5_xxl_average'}))
polite_p1_xxl = polite_p1_xxl_1.merge(polite_p1_xxl_2, how='left', left_index=True, right_index=True)
polite_p1_xxl = polite_p1_xxl[['p1_flant5_xxl_base', 'p1_flant5_xxl_man', 'p1_flant5_xxl_woman', 'p1_flant5_xxl_male', 'p1_flant5_xxl_female',
                                     'p1_flant5_xxl_white', 'p1_flant5_xxl_black', 'p1_flant5_xxl_asian',
                                     'p1_flant5_xxl_normal', 'p1_flant5_xxl_ordinary', 'p1_flant5_xxl_average']]

polite_p1_ul2_1 = (polite_p1_ul2_1[['a person', 'a white person', 'a black person', 'an Asian person']]
                    .rename(columns={'a person':'p1_flan_ul2_base',
                                     'a white person':'p1_flan_ul2_white',
                                     'a black person':'p1_flan_ul2_black',
                                     'an Asian person':'p1_flan_ul2_asian'}))
polite_p1_ul2_2 = (polite_p1_ul2_2
                      .rename(columns={'a man':'p1_flan_ul2_man',
                                       'a woman':'p1_flan_ul2_woman',
                                       'a male person':'p1_flan_ul2_male',
                                       'a female person':'p1_flan_ul2_female',
                                       'a normal person':'p1_flan_ul2_normal',
                                       'an ordinary person':'p1_flan_ul2_ordinary',
                                       'an average person':'p1_flan_ul2_average'}))
polite_p1_ul2 = polite_p1_ul2_1.merge(polite_p1_ul2_2, how='left', left_index=True, right_index=True)
polite_p1_ul2 = polite_p1_ul2[['p1_flan_ul2_base', 'p1_flan_ul2_man', 'p1_flan_ul2_woman', 'p1_flan_ul2_male', 'p1_flan_ul2_female',
                                     'p1_flan_ul2_white', 'p1_flan_ul2_black', 'p1_flan_ul2_asian',
                                     'p1_flan_ul2_normal', 'p1_flan_ul2_ordinary', 'p1_flan_ul2_average']]

polite_p2_xxl = (polite_p2_xxl
                    .rename(columns={'a person':'p2_flant5_xxl_base',
                                     'a man':'p2_flant5_xxl_man',
                                     'a woman':'p2_flant5_xxl_woman',
                                     'a male person':'p2_flant5_xxl_male',
                                     'a female person':'p2_flant5_xxl_female',
                                     'a white person':'p2_flant5_xxl_white',
                                     'a black person':'p2_flant5_xxl_black',
                                     'an asian person':'p2_flant5_xxl_asian',
                                     'a normal person':'p2_flant5_xxl_normal',
                                     'an ordinary person':'p2_flant5_xxl_ordinary',
                                     'an average person':'p2_flant5_xxl_average'}))

polite_p2_ul2 = (polite_p2_ul2
                    .rename(columns={'a person':'p2_flan_ul2_base',
                                     'a man':'p2_flan_ul2_man',
                                     'a woman':'p2_flan_ul2_woman',
                                     'a male person':'p2_flan_ul2_male',
                                     'a female person':'p2_flan_ul2_female',
                                     'a white person':'p2_flan_ul2_white',
                                     'a black person':'p2_flan_ul2_black',
                                     'an asian person':'p2_flan_ul2_asian',
                                     'a normal person':'p2_flan_ul2_normal',
                                     'an ordinary person':'p2_flan_ul2_ordinary',
                                     'an average person':'p2_flan_ul2_average'}))

polite_p3_xxl = (polite_p3_xxl
                    .rename(columns={'a person':'p3_flant5_xxl_base',
                                     'a man':'p3_flant5_xxl_man',
                                     'a woman':'p3_flant5_xxl_woman',
                                     'a male person':'p3_flant5_xxl_male',
                                     'a female person':'p3_flant5_xxl_female',
                                     'a white person':'p3_flant5_xxl_white',
                                     'a black person':'p3_flant5_xxl_black',
                                     'an asian person':'p3_flant5_xxl_asian',
                                     'a normal person':'p3_flant5_xxl_normal',
                                     'an ordinary person':'p3_flant5_xxl_ordinary',
                                     'an average person':'p3_flant5_xxl_average'}))

polite_p3_ul2 = (polite_p3_ul2
                    .rename(columns={'a person':'p3_flan_ul2_base',
                                     'a man':'p3_flan_ul2_man',
                                     'a woman':'p3_flan_ul2_woman',
                                     'a male person':'p3_flan_ul2_male',
                                     'a female person':'p3_flan_ul2_female',
                                     'a white person':'p3_flan_ul2_white',
                                     'a black person':'p3_flan_ul2_black',
                                     'an asian person':'p3_flan_ul2_asian',
                                     'a normal person':'p3_flan_ul2_normal',
                                     'an ordinary person':'p3_flan_ul2_ordinary',
                                     'an average person':'p3_flan_ul2_average'}))

polite_prompts_w = (polite
                       .merge(polite_p1_xxl, how='left', left_index=True, right_index=True)
                       .merge(polite_p1_ul2, how='left', left_index=True, right_index=True)
                       .merge(polite_p2_xxl, how='left', left_index=True, right_index=True)
                       .merge(polite_p2_ul2, how='left', left_index=True, right_index=True)
                       .merge(polite_p3_xxl, how='left', left_index=True, right_index=True)
                       .merge(polite_p3_ul2, how='left', left_index=True, right_index=True))
polite_prompts_w.to_csv("../results/polite_prompts_w.tsv", sep="\t", index=False)

## Add Tulu

In [ ]:
offensive_prompts_w = pd.read_table("offensive_prompts_w.tsv")
offensive_prompts = offensive_prompts_w[['label', 'p1_flant5_xxl_base', 'p1_flan_ul2_base',
                                         'p2_flant5_xxl_base', 'p2_flan_ul2_base',
                                         'p3_flant5_xxl_base', 'p3_flan_ul2_base' ]]
offensive_prompts['p1_tulu2_7b_base'] = (pd.read_csv("offensiveness_results_p1_tulu2_7b.csv").loc[:,'a person'])
offensive_prompts['p1_tulu2_13b_base'] = (pd.read_csv("offensiveness_results_p1_tulu2_13b.csv").loc[:,'a person'])
offensive_prompts['p2_tulu2_7b_base'] = (pd.read_csv("offensiveness_predictions_p2_tulu2_7b.csv").loc[:,'a person'].apply(find_rating_tulu))
offensive_prompts['p2_tulu2_13b_base'] = (pd.read_csv("offensiveness_predictions_p2_tulu2_13b.csv").loc[:,'a person'].apply(find_rating_tulu))
offensive_prompts['p3_tulu2_7b_base'] = (pd.read_csv("offensiveness_predictions_p3_tulu2_7b.csv").loc[:,'a person'].apply(find_rating_tulu))
offensive_prompts['p3_tulu2_13b_base'] = (pd.read_csv("offensiveness_predictions_p3_tulu2_13b.csv").loc[:,'a person'].apply(find_rating_tulu))

In [5]:
offensive_robust_corr = (offensive_prompts.corr().loc['p1_flant5_xxl_base':'p3_tulu2_13b_base', 'label']).reset_index()
offensive_robust_corr['dimension'] = "offensiveness"
offensive_robust_corr['model'] = ['flant5_xxl', 'flan_ul2', 'flant5_xxl', 'flan_ul2', 'flant5_xxl', 'flan_ul2',
                                  'tulu2_7b', 'tulu2_13b', 'tulu2_7b', 'tulu2_13b', 'tulu2_7b', 'tulu2_13b']
offensive_robust_corr['prompt'] = ['p1', 'p1', 'p2', 'p2', 'p3', 'p3', 'p1', 'p1', 'p2', 'p2', 'p3', 'p3']
del offensive_robust_corr['index']
offensive_robust_corr.rename(columns={'label':'corr'}, inplace=True)

In [ ]:
polite_prompts_w = pd.read_table("polite_prompts_w.tsv")
polite_prompts = polite_prompts_w[['label', 'p1_flant5_xxl_base', 'p1_flan_ul2_base',
                                         'p2_flant5_xxl_base', 'p2_flan_ul2_base',
                                         'p3_flant5_xxl_base', 'p3_flan_ul2_base' ]]
polite_prompts['p1_tulu2_7b_base'] = (pd.read_csv("politeness_results_p1_tulu2_7b.csv").loc[:,'a person'])
polite_prompts['p1_tulu2_13b_base'] = (pd.read_csv("politeness_results_p1_tulu2_13b.csv").loc[:,'a person'])
polite_prompts['p2_tulu2_7b_base'] = (pd.read_csv("politeness_predictions_p2_tulu2_7b.csv").loc[:,'a person'].apply(find_rating_tulu))
polite_prompts['p2_tulu2_13b_base'] = (pd.read_csv("politeness_predictions_p2_tulu2_13b.csv").loc[:,'a person'].apply(find_rating_tulu))
polite_prompts['p3_tulu2_7b_base'] = (pd.read_csv("politeness_predictions_p3_tulu2_7b.csv").loc[:,'a person'].apply(find_rating_tulu))
polite_prompts['p3_tulu2_13b_base'] = (pd.read_csv("politeness_predictions_p3_tulu2_13b.csv").loc[:,'a person'].apply(find_rating_tulu))

In [7]:
polite_robust_corr = (polite_prompts.corr().loc['p1_flant5_xxl_base':'p3_tulu2_13b_base', 'label']).reset_index()
polite_robust_corr['dimension'] = "politeness"
polite_robust_corr['model'] = ['flant5_xxl', 'flan_ul2', 'flant5_xxl', 'flan_ul2', 'flant5_xxl', 'flan_ul2',
                                  'tulu2_7b', 'tulu2_13b', 'tulu2_7b', 'tulu2_13b', 'tulu2_7b', 'tulu2_13b']
polite_robust_corr['prompt'] = ['p1', 'p1', 'p2', 'p2', 'p3', 'p3', 'p1', 'p1', 'p2', 'p2', 'p3', 'p3']
del polite_robust_corr['index']
polite_robust_corr.rename(columns={'label':'corr'}, inplace=True)

In [8]:
robust_corr = pd.concat([offensive_robust_corr, polite_robust_corr])
robust_corr.to_csv("../results/corr_prompt_l.csv", index=False)

# Statistic Analysis Table - Gap

## Offensiveness

In [14]:
offensive = pd.read_table("../results/offensive_results_w2.tsv")
offensive_score = offensive.loc[:, 'man_score':'asian_score']
offensive_score.columns = ['man', 'woman', 'white', 'black', 'asian']

models = ['flant5_xxl', 'flan_ul2', 'tulu2_7b', 'tulu2_13b', 'gpt35', 'gpt4', 'llama31_8b', 'mistral03_7b', 'qwen25_7b']
offensive_gap = pd.DataFrame({'id':[], 'model':[], 'label':[], 'prompt':[], 'gap':[]})

for model in models:
    base_gap = (-offensive_score.sub(offensive[f'{model}_base'], axis=0))
    base_gap['id'] = offensive['instance_id']
    base_gap = base_gap.melt(id_vars='id', var_name='label', value_name='gap')
    base_gap['model'] = model
    base_gap['prompt'] = 0

    identity_gap = offensive.loc[:, f'{model}_male':f'{model}_asian']
    identity_gap.columns = ['man', 'woman', 'white', 'black', 'asian']
    identity_gap = identity_gap.sub(offensive_score, axis=0)
    identity_gap['id'] = offensive['instance_id']
    identity_gap = identity_gap.melt(id_vars='id', var_name='label', value_name='gap')
    identity_gap['model'] = model
    identity_gap['prompt'] = 1

    offensive_gap = pd.concat([offensive_gap, base_gap, identity_gap], ignore_index=True)

offensive_gap.to_csv("../results/offensive_gap_l2.tsv", sep="\t", index=False)

## Politeness

In [15]:
polite = pd.read_table("../results/polite_results_w2.tsv")
polite_score = polite.loc[:, 'man_score':'asian_score']
polite_score.columns = ['man', 'woman', 'white', 'black', 'asian']

models = ['flant5_xxl', 'flan_ul2', 'tulu2_7b', 'tulu2_13b', 'gpt35', 'gpt4', 'llama31_8b', 'mistral03_7b', 'qwen25_7b']
polite_gap = pd.DataFrame({'id':[], 'model':[], 'label':[], 'prompt':[], 'gap':[]})

for model in models:
    base_gap = (-polite_score.sub(polite[f'{model}_base'], axis=0))
    base_gap['id'] = polite['instance_id']
    base_gap = base_gap.melt(id_vars='id', var_name='label', value_name='gap')
    base_gap['model'] = model
    base_gap['prompt'] = 0

    identity_gap = polite.loc[:, f'{model}_male':f'{model}_asian']
    identity_gap.columns = ['man', 'woman', 'white', 'black', 'asian']
    identity_gap = identity_gap.sub(polite_score, axis=0)
    identity_gap['id'] = polite['instance_id']
    identity_gap = identity_gap.melt(id_vars='id', var_name='label', value_name='gap')
    identity_gap['model'] = model
    identity_gap['prompt'] = 1

    polite_gap = pd.concat([polite_gap, base_gap, identity_gap], ignore_index=True)

polite_gap.to_csv("../results/polite_gap_l2.tsv", sep="\t", index=False)